# Aspect Based Sentiment Analysis

Getting sentiment analysis on aspects of headphones. 

https://medium.com/nlplanet/quick-intro-to-aspect-based-sentiment-analysis-c8888a09eda7

https://huggingface.co/yangheng/deberta-v3-base-absa-v1.1